In [1]:
from pyspark.sql import SparkSession, Row
from operator import add
from pyspark.sql.functions import explode_outer

In [2]:
# import pyspark.sql.functions as f
# # Read data
# df = spark.read.json('/home/ubuntu/processed_data.json')

# # Convert the profane words to a DataFrame
# PROFANE_WORDS = ['omorashi', 'blowjob', 'scat', 'snowballing', 'shitty', 'tits', 'tribadism', 'strap on', 'hot carl', 'wank', 'twat', 'zoophilia', 'spic', 'homoerotic', 'pisspig', 'jailbait', 'bastardo', 'shemale', 'xx', 'butthole', 'yaoi', 'kinkster', 'bastard', 'faggot', 'shit', 'frotting', 'beaner', 'baby juice', 'poontang', 'sultry women', 'female squirting', 'nigga', 'strapon', 's&m', 'gay sex', 'nudity', 'darkie', 'vibrator', 'genitals', 'male squirting', 'birdlock', 'date rape', 'double penetration', 'sucks', 'tub girl', 'rapist', 'urophilia', 'fisting', 'blumpkin', 'piece of shit', 'poop chute', 'nigger', 'poon', 'doggiestyle', 'rimjob', 'splooge', 'bullshit', 'ponyplay', 'yiffy', 'dvda', 'alabama hot pocket', 'beaver lips', 'double dong', 'donkey punch', 'slanteye', 'eat my ass', 'babeland', 'wetback', 'spread legs', 'bondage', 'nambla', 'feltch', 'masturbate', 'juggs', 'kinbaku', 'twinkie', 'bollocks', 'fucking', 'pedobear', 'beaners', 'dendrophilia', 'erotism', 'pussy', 'barenaked', 'queaf', 'porn', 'twink', 'knobbing', 'neonazi', 'raping', 'shitblimp', 'coon', 'blonde action', 'reverse cowgirl', 'spooge', 'girl on', 'acrotomophilia', 'porno', 'creampie', 'piss pig', 'doggie style', 'futanari', 'femdom', 'hentai', 'towelhead', 'god damn', 'topless', 'bangbros', 'sodomy', 'nig nog', 'bulldyke', 'black cock', 'clusterfuck', 'bukkake', 'felch', 'jigaboo', 'white power', 'boob', 'guro', 'rectum', 'doggy style', 'kinky', 'sexy', 'coprolagnia', 'undressing', 'muffdiving', 'blow job', 'dolcett', 'blonde on blonde action', 'shibari', 'punany', 'ass', 'tushy', 'anal', 'hooker', 'humping', 'dildo', 'wrinkled starfish', 'ball gag', 'gang bang', 'g-spot', 'pleasure chest', 'dry hump', 'camgirl', 'brown showers', 'anilingus', 'sodomize', 'bestiality', 'coons', 'carpetmuncher', 'schlong', 'auto erotic', 'lemon party', 'goatcx', 'intercourse', 'hand job', '2g1c', 'nude', 'domination', 'boner', 'boobs', 'make me come', 'kike', 'group sex', 'bimbos', 'how to kill', 'jail bait', 'fellatio', 'circlejerk', 'menage a trois', 'style doggy', 'queef', 'goodpoop', 'pissing', 'alaskan pipeline', 'panty', 'girls gone wild', 'tainted love', 'rosy palm and her 5 sisters', 'muff diver', 'autoerotic', 'goregasm', 'jack off', 'orgasm', 'dingleberry', 'bung hole', 'handjob', 'dirty sanchez', 'quim', 'mound of venus', 'pole smoker', 'rosy palm', 'big tits', 'big knockers', 'shrimping', 'threesome', 'poopchute', 'grope', 'strip club', 'two girls one cup', 'honkey', 'motherfucker', 'tongue in a', 'strappado', 'asshole', 'cornhole', 'goatse', 'jerk off', 'blue waffle', 'violet wand', 'deep throat', 'butt', 'dog style', 'camslut', 'dick', 'deepthroat', 'barely legal', 'fucktards', 'fuckin', 'dp action', 'arsehole', 'wrapping men', 'dominatrix', 'santorum', 'swastika', 'tight white', 'big breasts', 'jiggerboo', 'anus', 'upskirt', 'cunnilingus', 'titty', 'buttcheeks', 'doggystyle', 'bareback', 'playboy', 'nipple', 'camwhore', 'mr hands', 'big black', 'fingering', 'how to murder', 'brunette action', 'milf', 'cock', 'sadism', 'foot fetish', 'giant cock', 'phone sex', 'huge fat', 'penis', 'vagina', 'ejaculation', 'voyeur', 'shaved beaver', 'shota', 'pedophile', 'camel toe', 'jiggaboo', 'dommes', 'carpet muncher', 'busty', 'rimming', 'paedophile', 'suicide girls', 'orgy', 'nsfw images', 'escort', 'booty call', 'rape', 'bastinado', 'cunt', 'nipples', 'tied up', 'scissoring', 'titties', 'taste my', 'sex', 'spunk', 'cum', 'semen', 'baby batter', 'prince albert piercing', 'one cup two girls', 'urethra play', 'ecchi', 'fingerbang', 'missionary position', 'tubgirl', 'vorarephilia', 'jizz', 'nimphomania', 'wet dream', 'xxx', 'nympho', 'ball kicking', 'erotic', 'tit', 'golden shower', 'goo girl', 'blow your load', 'one guy one jar', 'ball sucking', 'assmunch', 'incest', 'pornography', 'coprophilia', 'lolita', 'poof', 'nymphomania', 'eunuch', 'cocks', 'fudge packer', 'pegging', 'pthc', 'hardcore', 'paki', 'jelly donut', 'tosser', 'panties', 'shaved pussy', 'bdsm', 'sexo', 'gokkun', 'venus mound', 'chocolate rosebuds', 'footjob', 'pubes', 'bitches', 'beaver cleaver', 'clitoris', '2 girls 1 cup', 'hard core', 'swinger', 'skeet', 'clover clamps', 'dirty pillows', 'ball licking', 'raging boner', 'ball sack', 'leather restraint', 'hot chick', 'throating', 'rusty trombone', 'splooge moose', 'bullet vibe', 'bunghole', 'fecal', 'suck', 'leather straight jacket', 'octopussy', 'vulva', 'tranny', 'slut', 'fuck buttons', 'snatch', 'girl on top', 'negro', 'dingleberries', 'bbw', 'cumming', 'tea bagging', 'ball gravy', 'nawashi', 'cleveland steamer', 'daterape', 'apeshit', 'lovemaking', 'raghead', 'fudgepacker', 'yellow showers', 'clit', 'smut', 'fuck', 'figging', 'bitch']
# df_pw = spark.createDataFrame([Row(arr=PROFANE_WORDS)])
# df_pw.groupBy().pivot('arr').agg(f.first('Value')).show()

# # Tokenize the text
# tokenizer = Tokenizer(inputCol='parsed_text', outputCol="tokens")
# tokenizer.transform(df).select('tokens')


# # OLD METHOD
# lines_testfile = spark.read.text('testfile.txt').rdd.map(lambda r: r[0])
# print(lines_testfile.take(5))


# # NEW METHOD
# #rdd_data = spark.read.json('processed_data.json')
# exploded = rdd_data.select('url', explode_outer(rdd_data.neighbor_urls).alias("neighbour_url"))
# lines_json = exploded.rdd.map(lambda x: x)
# print(lines_json.take(5))


# links = lines_json.map(lambda urls: parseNeighbors(urls)).distinct().groupByKey().cache()
# #print(links.take(5))
# #links.take(5)

In [3]:
# Code copied from: https://github.com/apache/spark/blob/master/examples/src/main/python/pagerank.py
def computeContribs(urls, rank):
    """Calculates URL contributions to the rank of other URLs."""
    num_urls = len(urls)
    for url in urls:
        yield (url, rank / num_urls)

def parseNeighbors(urls):
    """Parses a urls pair string into urls pair."""
    return urls[0], urls[1]


def pageRank(file, iters = 10):
    """Naive PageRank algorithm."""
    
    # Initialize the spark context.
    spark = SparkSession\
        .builder\
        .appName("PythonPageRank")\
        .getOrCreate()

    # Loads in input file. It should be in format of:
    #     URL         neighbor URL
    #     URL         neighbor URL
    #     URL         neighbor URL
    #     ...


    # Read the data
    df = spark.read.json(file)

    # Explode to get the right format.
    exploded_df = df.select('url', explode_outer(df.neighbor_urls).alias("neighbour_url"))
    lines = exploded_df.rdd.map(lambda x: x)

    # Loads all URLs from input file and initialize their neighbors.
    links = lines.map(lambda urls: parseNeighbors(urls)).distinct().groupByKey().cache()

    # Loads all URLs with other URL(s) link to from input file and initialize ranks of them to one.
    ranks = links.map(lambda url_neighbors: (url_neighbors[0], 1.0))


    # Calculates and updates URL ranks continuously using PageRank algorithm.
    for iteration in range(int(iters)):
        # Calculates URL contributions to the rank of other URLs.
        contribs = links.join(ranks).flatMap(lambda url_urls_rank: computeContribs(
            url_urls_rank[1][0], url_urls_rank[1][1]  # type: ignore[arg-type]
        ))
        # Re-calculates URL ranks based on neighbor contributions.
        ranks = contribs.reduceByKey(add).mapValues(lambda rank: rank * 0.85 + 0.15)

    # Collects all URL ranks and dump them to console.
    for (link, rank) in ranks.collect():
        print("%s has rank: %s." % (link, rank))

    spark.stop()

pageRank('/home/ubuntu/processed_data.json')

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/04/27 13:29:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


http://gmpg.org has rank: 0.20930496183490793.
http://fly.carlosparaglide.com has rank: 0.20930496183490793.
https://api.w.org has rank: 0.20930496183490793.
